# Partie 2

In [2]:
import requests

In [ ]:
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
from tensorflow.keras.optimizers import Adam
from transformers import AdamWeightDecay
import tensorflow as tf
import requests
import numpy as np
 
url = "https://storage.googleapis.com/tensorflow/tf-keras-datasets/imdb.npz"
response = requests.get(url)
 
with open("imdb.npz", "wb") as f:
    f.write(response.content)
 
with np.load("imdb.npz", allow_pickle=True) as data:
    x_train, y_train = data["x_train"], data["y_train"]
    x_test, y_test = data["x_test"], data["y_test"]
 
 
# Tokenisation avec DistilBERT
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
 
# Encoder les données d'entraînement et de test
x_train_enc = tokenizer([" ".join(map(str, review)) for review in x_train], padding=True, truncation=True, return_tensors="tf")
x_test_enc = tokenizer([" ".join(map(str, review)) for review in x_test], padding=True, truncation=True, return_tensors="tf")
 
 
# Charger le modèle DistilBERT pour la classification de séquences
model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)
 
 
# Utiliser l'optimiseur AdamWeightDecay
optimizer = AdamWeightDecay(learning_rate=5e-5)
 
# Fonction de perte
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
 
# Compiler le modèle
model.compile(optimizer=optimizer, loss=loss_fn, metrics=["accuracy"])
 
# Créer des datasets TensorFlow pour l'entraînement et la validation
train_dataset = tf.data.Dataset.from_tensor_slices((dict(x_train_enc), y_train)).batch(32)
test_dataset = tf.data.Dataset.from_tensor_slices((dict(x_test_enc), y_test)).batch(32)
 
# Entraîner le modèle
model.fit(train_dataset, epochs=3, validation_data=test_dataset)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-06 12:35:27.108104: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or